# Forma AI SQL Take Home Assignment

The purpose of this notebook is to answer the questions laid out by Forma AI for the data analyst role.

As a part of the assignment Forma AI has provided a SQLite3 database and 5 questions. The questions will be reiterated in this notebook but more information can be found in the README file.

## The database

The SQLite3 database contains 3 tables.

The first table `transactions`, contains details about each product that a customer has purchased. A transaction is labeled by a 'trans_id' and can contain multiple products. Here are the columns in the `transactions` table:
   * `trans_id`: the transaction id
   * `trans_dt`: the date that the transaction took place (this was not originally included in the provided README file)
   * `cust_id`: the customer id
   * `prod_id`: the product id
   * `item_qty`: the quantity of the product that is being purchased
   * `item_price`: the per unit price of the product (NOTE: the total revenue
     for a product is `item_qty * item_price`)

The second table `products`, contains details about each product. The columns in the `products` table are:
   * `prod_id`: the product id (same meaning as in `transactions`)
   * `prod_name`: the product name
   * `brand`: the brand of the product
   * `category`: the category of the product
   
Finally the third table `segments`, contains the history of each customer and which market segment they belong to. Segmentation is calculated periodically for current customers and are appended to this table. The most recent segment is labeled in the `active-flag` column by a `Y`. The columns in this table are:
   * `cust_id`: the customer id (same meaning as in `transactions`)
   * `seg_name`: the segment of this customer
   * `update_at`: the date when this segment was updated (in the original text file this was labeled as `update_dt` we later discovered the column was actually called `update_at`)
   * `active_flag`: whether or not this segment is the active segment for this customer


## Reading the database

Before we can begin answering the questions let us import the `sqlite3` library and connecting to the database.

In [1]:
import sqlite3

conn = sqlite3.connect('sample.db')

sample_query = 'SELECT * FROM transactions;'
sample_pull = conn.execute(sample_query).fetchmany(5)
print(sample_pull)


[(1, '2016-01-02 10:06:00', 9085146, 223029, 1, 42.99), (2, '2016-01-02 10:30:00', 1215814, 252270, 1, 103.95), (2, '2016-01-02 10:30:00', 1215814, 260383, 1, 74.99), (4, '2016-01-02 11:33:00', 18511160, 269119, 1, 51.99), (4, '2016-01-02 11:33:00', 18511160, 411162, 1, 59.99)]


When pulling our sample query from the `transactions` table we notice another column that was not in the provided README file. Let us pull the column headers of the `transactions` table.

In [2]:
print(conn.execute('PRAGMA TABLE_INFO(transactions);').fetchall())

[(0, 'trans_id', 'INTEGER', 0, None, 0), (1, 'trans_dt', 'TIMESTAMP', 0, None, 0), (2, 'cust_id', 'INTEGER', 0, None, 0), (3, 'prod_id', 'INTEGER', 0, None, 0), (4, 'item_qty', 'INTEGER', 0, None, 0), (5, 'item_price', 'REAL', 0, None, 0)]



We see that this column is named `trans_dt`. Let us update our intro and begin answering the questions.

## Question 1

**Find the current active segment for each customer sorted by the segment
   update date.  The output should contain three columns: `cust_id`,
   `seg_name`, `updated_at`**
   
Let's assume that the `updated_at` column should be ordered in descending order. We will be using the `sqlite3` library.

In [3]:
#Pulling the data using the sqlite3 library

query_1 = '''
SELECT cust_id,
       seg_name,
       update_dt AS updated_at
FROM segments
WHERE active_flag = Y
ORDER BY updated_at DESC;
'''

question_1 = conn.execute(query_1).fetchall()
print(question_1[:3])

OperationalError: no such column: update_dt


When we tried to run the query we got an error saying there was no such column `update_dt`. Let's query the column headers of the `segments` table to dive deeper.

In [4]:
print(conn.execute('PRAGMA TABLE_INFO(segments);').fetchall())

[(0, 'cust_id', 'INTEGER', 0, None, 0), (1, 'seg_name', 'TEXT', 0, None, 0), (2, 'update_at', 'TIMESTAMP', 0, None, 0), (3, 'active_flag', 'TEXT', 0, None, 0)]



We see that the column column describing when the segments were updated is called `update_at` not `update_dt`. Let us update our query with this new information.

In [5]:
query_1 = '''
SELECT cust_id,
       seg_name,
       update_at AS updated_at
FROM segments
WHERE active_flag = 'Y'
ORDER BY updated_at DESC;
'''

question_1 = conn.execute(query_1).fetchall()
print(question_1[:3])

[(12064, 'INFREQUENT', '2016-06-01 00:00:00'), (106895, 'INFREQUENT', '2016-06-01 00:00:00'), (298867, 'ONE-OFFS', '2016-06-01 00:00:00')]



When working with data we I would be using `Pandas` to turn a pull into a dataframe to easily clean and analyze the data. Let's do that now.

In [6]:
import pandas as pd

question_1_df = pd.read_sql_query(query_1, conn)
question_1_df

,cust_id,seg_name,updated_at
0,12064,INFREQUENT,2016-06-01 00:00:00
1,106895,INFREQUENT,2016-06-01 00:00:00
2,298867,ONE-OFFS,2016-06-01 00:00:00
3,449146,INFREQUENT,2016-06-01 00:00:00
4,689576,ONE-OFFS,2016-06-01 00:00:00
...,...,...,...
1317,11259901,VIP,2014-03-01 00:00:00
1318,5764946,VIP,2014-01-01 00:00:00
1319,8042053,VIP,2014-01-01 00:00:00
1320,14075295,VIP,2014-01-01 00:00:00


## Question 2

**For each product purchased between Jan 2016 and May 2016 (inclusive), find
   the number of distinct transactions.  The output should contain `prod_id`,
   `prod_name` and distinct transaction columns.**

For this question we will use an inner join on the `transactions` table and the `products` table assuming that all product ids have an associated product name. We will also sort by the `count` column in descending order.

In [7]:
query_2 = '''
SELECT t.prod_id,
       p.prod_name,
       SUM(item_qty) AS count
FROM transactions t
INNER JOIN products p ON t.prod_id = p.prod_id
WHERE trans_dt BETWEEN '2016-01-01' AND '2016-05-31'
GROUP BY t.prod_id
ORDER BY COUNT DESC;
'''

question_2_df = pd.read_sql_query(query_2, conn)
question_2_df

,prod_id,prod_name,count
0,226594814,Product 226594814,10
1,215734505,Product 215734505,10
2,138262084,Product 138262084,9
3,261231,Product 261231,9
4,220108,Product 220108,9
...,...,...,...
1804,211200,Product 211200,1
1805,209999,Product 209999,1
1806,209732,Product 209732,1
1807,207344,Product 207344,1


## Question 3

**Find the most recent segment of each customer as of 2016-03-01.
   *Hint*: You cannot simply use `active_flag` since that is as of the current
   date *not* 2016-03-01.  The output should contain the `cust_id`, `seg_name`
   and `update_at`  columns and should have at most one row per customer.**
   
Once again we will read our query into a dataframe. Since the segment updates just get appended to the table we can can find the max value of the `update_at` column for each `cust_id` as long as we filter the data for dates before March 2016.

In [21]:
query_3 = '''
SELECT cust_id,
       seg_name,
       MAX(update_at) AS update_at
FROM segments
WHERE update_at < '2016-03-01'
GROUP BY cust_id;
'''

question_3_df = pd.read_sql_query(query_3, conn)
question_3_df

,cust_id,seg_name,update_at
0,4402,ONE-OFFS,2016-02-01 00:00:00
1,11248,LOYAL,2016-02-01 00:00:00
2,12064,INFREQUENT,2016-02-01 00:00:00
3,15088,ONE-OFFS,2016-02-01 00:00:00
4,66706,ONE-OFFS,2016-02-01 00:00:00
...,...,...,...
995,21233469,NEW,2016-02-01 00:00:00
996,21233549,NEW,2016-02-01 00:00:00
997,21233596,NEW,2016-02-01 00:00:00
998,21233911,NEW,2016-02-01 00:00:00


## Question 4

**Find the most popular category (by revenue) for each active segment.
   *Hint*: The current (most up to date) active segment is specified by `active_flag = 'Y'` column in the segments table.**
   
For this question we need to merge all three tables together. When we merge the `transactions` and `segments` we can figure out which segment the customer belongs to. When we merge the `transactions` and `products` table we can determine which category the item belongs to. We also need to create the `revenue` column multiplying the `item_qty` and `item_price` columns.

Since we will be using multiple `WITH` statements within our query we should test these out before creating our bigger query. The first one will be using the `transactions` table to find revenue.

In [34]:
query_sales_revenue = '''
SELECT cust_id,
       prod_id,
       SUM(item_qty * item_price) AS revenue
FROM transactions
GROUP BY prod_id
ORDER BY cust_id; 
'''
#We added the ORDER BY function to ensure that if a customer bought mulitple items that they would show up in their own row


revenue_df = pd.read_sql_query(query_sales_revenue, conn)
revenue_df

,cust_id,prod_id,revenue
0,4402,285204,24.99
1,11248,495237,24.99
2,11248,113495842,34.99
3,12064,238891,20.99
4,12064,192007896,218.97
...,...,...,...
1858,21233469,166838318,72.99
1859,21233549,268780,71.99
1860,21233596,449428,88.99
1861,21233911,257304,13.99


In [24]:
query_active_segments = '''
SELECT cust_id,
        seg_name
FROM segments
WHERE active_flag = 'Y';
'''

active_segments_df = pd.read_sql_query(query_active_segments, conn)
active_segments_df

,cust_id,seg_name
0,4402,ONE-OFFS
1,11248,LOYAL
2,12064,INFREQUENT
3,12064,INFREQUENT
4,15088,ONE-OFFS
...,...,...
1317,21233469,NEW
1318,21233549,NEW
1319,21233596,NEW
1320,21233911,NEW


In [37]:
query_4 = '''
SELECT s.seg_name AS seg_name,
       p.category AS category,
       MAX(t.revenue) as revenue
FROM (SELECT cust_id,
        prod_id,
        SUM(item_qty * item_price) AS revenue
      FROM transactions
      GROUP BY prod_id) t
INNER JOIN (SELECT cust_id,
                seg_name
            FROM segments
            WHERE active_flag = 'Y') s ON t.cust_id = s.cust_id
INNER JOIN products p ON t.prod_id = p.prod_id
GROUP BY s.seg_name;
'''

question_4_df = pd.read_sql_query(query_4, conn)
question_4_df

DatabaseError: Execution failed on sql '
SELECT s.seg_name AS seg_name,
       p.category AS category,
       MAX(SUM(t.revenue)) as revenue
FROM (SELECT cust_id,
        prod_id,
        SUM(item_qty * item_price) AS revenue
      FROM transactions
      GROUP BY prod_id) t
INNER JOIN (SELECT cust_id,
                seg_name
            FROM segments
            WHERE active_flag = 'Y') s ON t.cust_id = s.cust_id
INNER JOIN products p ON t.prod_id = p.prod_id
GROUP BY s.seg_name;
': misuse of aggregate function SUM()